In [ ]:
!pip install boto3
!pip install awscli
!pip install --upgrade openai
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 5.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 30.4 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: Py

In [ ]:
import boto3
import openai
from neo4j import GraphDatabase


In [ ]:
!aws configure

kendra_index_id = input("Your Kendra index id: ")

GPT_APIKEY = input("Your GPT-3 API Key: ")

url = input("Your Doctor.ai backend url: ")
neo4j_username = input("Your Doctor.ai username: ")
neo4j_password = input("Your Doctor.ai password: ")
driver = GraphDatabase.driver(url, auth=(neo4j_username, neo4j_password))

In [ ]:
kendra = boto3.client("kendra", region_name='us-east-1')

def get_kendra(query):
    
    response = kendra.query(
            QueryText = query,
            IndexId = kendra_index_id)      

    answer = ""
    for query_result in response["ResultItems"]:

        
        if query_result["Type"]=="QUESTION_ANSWER":
            answer_text = query_result["DocumentExcerpt"]["Text"]
            return answer_text

        elif query_result["Type"]=="ANSWER":
            answer_text = query_result["DocumentExcerpt"]["Text"]
            content = ""
            for line in answer_text.split("\n"):
                line = line.strip()
                if line != "":
                    content += " " + line
            answer = content
    return answer




In [ ]:
get_kendra("Which drugs are used to treat COVID?")

'Remdesivir, nirmatrelvir and ritonavir'

In [ ]:
get_kendra("What is pulmonary embolism?")

'A pulmonary embolism is a blocked blood vessel in lungs. It can be life-threatening if not treated quickly.'

In [ ]:
openai.api_key = GPT_APIKEY

def get_GPT_3 (query, temp):

  response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=query,
    temperature=temp,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )

  return response.choices[0].text.lstrip()



In [ ]:
training_text = """
#How many times did patient id_1 visit the ICU?
MATCH (p:Patient)-[:HAS_STAY]->(v:PatientUnitStay) WHERE p.patient_id =~ '(?i)id_1' RETURN COUNT(v)

#When did patient id_1 visit the ICU?
MATCH (p:Patient)-[:HAS_STAY]->(v:PatientUnitStay) WHERE p.patient_id =~ '(?i)id_1' RETURN v.hospitaldischargeyear

#What was the diagnosis of patient id_1's visit?; Why did patient id_1 visit the ICU?; What was the cause for patient id_1's visit?
MATCH (p:Patient)-[:HAS_STAY]->()-[:HAS_DIAG]->()-[:IS_DISEASE]->(d:Disease) WHERE p.patient_id =~ '(?i)id_1' RETURN d.name

#Which drug treats COVID-19?; Which kind of compound treats COVID-19?
MATCH (c:Compound)-[:treats]->(d:Disease) WHERE d.name =~ '(?i)COVID-19' RETURN c.name

#Which pathogen causes COVID-19?; What is the disease agent for COVID-19?; Which organism causes COVID-19?
MATCH (o:Pathogen)-[:causes]->(d:Disease) WHERE d.name =~ '(?i)COVID-19' RETURN o.name

#Which gene causes Christianson syndrome?
MATCH (g:Gene)-[r1:associates]->(d:Disease) WHERE d.name =~ '(?i)Christianson syndrome' RETURN g.name

#Tell me something about the disease named "Christianson syndrome"
MATCH (d:Disease) WHERE d.name =~ '(?i)Christianson syndrome' RETURN d.description

#I have Dyspepsia, Hiccup and Edema. What can be the cause of this?
MATCH (s1:Symptom) <-[:presents]- (d:Disease) WHERE s1.name =~ '(?i)Dyspepsia'  MATCH (s2:Symptom) <-[:presents]- (d:Disease) WHERE s2.name =~ '(?i)Hiccup'  MATCH (s3:Symptom) <-[:presents]- (d:Disease) WHERE s3.name =~ '(?i)Edema' RETURN d.name

#what kinds of side effects do Doxepin have?
MATCH (d:Compound)-[:causes]->(s:\`Side Effect\`) WHERE d.name =~ '(?i)Doxepin' RETURN s.name

#what functions does the gene PCBD1 have?
MATCH (g:Gene)-[:participates]->(f:\`Molecular Function\`) WHERE g.name =~ '(?i)PCBD1' RETURN f.name

#which kinds of cancers can be found in frontal sinus?; Which tumors can you find in frontal sinus?
MATCH (d:Disease)-[:localizes]->(a:Anatomy) WHERE a.name =~ '(?i)frontal sinus' AND (d.name CONTAINS "cancer" OR d.disease_category = "Cancer") RETURN DISTINCT(d.name)

#ELI5 what is lung cancer; Explain lung cancer in simple terms; Explain lung cancer in plain English
ELI5 MATCH (d:Disease) WHERE d.name =~ '(?i)lung cancer' RETURN d.description

#"""

In [ ]:
def get_cypher(query):
    prompt = training_text + query

    cypher = get_GPT_3(prompt, 0)

    return cypher

#cypher = get_cypher("What is small cell lung cancer?")
#print (cypher)

In [ ]:
def get_doctorai(query):
    cypher = get_cypher(query)

    with driver.session() as session:
        result = session.run(cypher)
        return ", ".join(result.value())



In [ ]:
answer_doctorai = get_doctorai("Which drugs are used to treat COVID-19?")
answer_doctorai

'Etesevimab, Nirmatrelvir and ritonavir, Sotrovimab, Elasomeran, Tozinameran, Molnupiravir, COVID-19 vaccine, Bamlanivimab, Remdesivir, Tixagevimab and cilgavimab'

In [ ]:
from typing import List
import ast

def get_consensus(statements: List[str], weights: List[int], my_query):
    statements_str = "\n".join([f"{i} | {y}" for i, y in enumerate(statements)])
    #print (statements_str)

    ensemble_prompt = f"""
You have three answers to the same question.

If the answer contains multiple items, write the statement id and convert the items into a list like this
Statements:
0 | They used A, B.
1 | B was their tools.
2 | They chose C plus A.

Question:
What do they use?

Answers:
The answers are items
0 | ["A", "B"]
1 | ["B"]
2 | ["A", "C"]

If they are statements without a list of items, unify them into a coherent statement like this
Statements:
0 | This medicine XYZ is used to treat lung cancer.
1 | XYZ is developed by the company ABC.
2 | The drug XYZ is approved by the FDA.

Question:
Explain the drug XYZ.

Answers:
The answer is a statement
The company ABC has developed the drug XYZ to treat lung cancer. XYZ has reveiced the FDA approval.


Statements:
{statements_str}

Question:
{my_query}

Answers:
"""
    
    result = get_GPT_3 (ensemble_prompt, 0)
    #print (ensemble_prompt)
    print (result)
    
    container = {}
    is_list = True
    for line in result.split("\n"):
        line = line.strip()
        if len(line) > 0:
            if line == "The answers are items":
                is_list = True
                continue
            elif line == "The answer is a statement":
                is_list = False
                continue
            
            if is_list == True:
                if "|" in line:
                    fields = line.split("|")

                    id = int(fields[0].strip())
                    items = ast.literal_eval(fields[1].strip())
                    
                    for item in items:
                        if item not in container:
                            container[item] = 0
                        container[item] += weights[id]
            else:
                container[line] = sum(weights)
    
    return container


bots = ["doctor.ai", "kendra", "GPT-3"]
statements = ["remdesivir, nirmatrelvir and ritonavir", "I used hydroxychloroquin to treat COVID", "remdesivir, nirmatrelvir and ritonavir have been successfully used against COVID-19"]
weights = [9, 6, 5]

print (get_consensus(statements, weights, "Which drugs are used to treat COVID?"))

The answers are items
0 | ["remdesivir", "nirmatrelvir", "ritonavir"]
1 | ["hydroxychloroquin"]
2 | ["remdesivir", "nirmatrelvir", "ritonavir"]
{'remdesivir': 14, 'nirmatrelvir': 14, 'ritonavir': 14, 'hydroxychloroquin': 6}


In [ ]:
my_query = "What is small cell lung cancer?"

In [ ]:
answer_kendra = get_kendra(my_query)
answer_kendra

'Small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.'

In [ ]:
answer_doctorai = get_doctorai(my_query)
answer_doctorai

'Lung cancer is a leading cause of cancer death among men and women in industrialized countries. Small cell lung carcinoma (SCLC) is a highly aggressive neoplasm, which accounts for approximately 25% of all lung cancer cases. Molecular mechanisms altered in SCLC include induced expression of oncogene, MYC, and loss of tumorsuppressor genes, such as p53, PTEN, RB, and FHIT. The overexpression of MYC proteins in SCLC is largely a result of gene amplification. Such overexpression leads to more rapid proliferation and loss of terminal differentiation. Mutation or deletion of p53 or PTEN can lead to more rapid proliferation and reduced apoptosis. The retinoblastoma gene RB1 encodes a nuclear phosphoprotein that helps to regulate cell-cycle progression. The fragile histidine triad gene FHIT encodes the enzyme diadenosine triphosphate hydrolase, which is thought to have an indirect role in proapoptosis and cell-cycle control.'

In [ ]:
answer_gpt_3 = get_GPT_3(my_query, 0.8)
answer_gpt_3

'Small cell lung cancer (SCLC) is an aggressive form of lung cancer that affects a small number of cells in the lung. It is one of two main types of lung cancer, the other being non-small cell lung cancer (NSCLC). It is much less common than NSCLC and tends to spread quickly, often to other parts of the body. Symptoms of SCLC can include coughing, chest pain, shortness of breath, and weight loss. Treatment typically involves a combination of chemotherapy, radiation therapy, and surgery.'

In [ ]:
bots = ["doctor.ai", "kendra", "GPT-3"]
statements = [answer_doctorai, answer_kendra, answer_gpt_3]
weights = [9, 6, 5]

for b, s in zip(bots, statements):
    print (f"{b}: {s}")

print (get_consensus(statements, weights, my_query))

doctor.ai: Lung cancer is a leading cause of cancer death among men and women in industrialized countries. Small cell lung carcinoma (SCLC) is a highly aggressive neoplasm, which accounts for approximately 25% of all lung cancer cases. Molecular mechanisms altered in SCLC include induced expression of oncogene, MYC, and loss of tumorsuppressor genes, such as p53, PTEN, RB, and FHIT. The overexpression of MYC proteins in SCLC is largely a result of gene amplification. Such overexpression leads to more rapid proliferation and loss of terminal differentiation. Mutation or deletion of p53 or PTEN can lead to more rapid proliferation and reduced apoptosis. The retinoblastoma gene RB1 encodes a nuclear phosphoprotein that helps to regulate cell-cycle progression. The fragile histidine triad gene FHIT encodes the enzyme diadenosine triphosphate hydrolase, which is thought to have an indirect role in proapoptosis and cell-cycle control.
kendra: Small cell lung cancer is a disease in which mali

In [ ]:
my_query_2 = "Which drugs are used to treat COVID?"

In [ ]:
answer_doctorai = get_doctorai(my_query_2)

answer_gpt_3 = get_GPT_3(my_query_2, 0.8)

answer_kendra = get_kendra(my_query_2)



In [ ]:
bots = ["doctor.ai", "kendra", "GPT-3"]
statements = [answer_doctorai.lower(), answer_kendra.lower(), answer_gpt_3.lower()]
weights = [9, 6, 5]

for b, s in zip(bots, statements):
    print (f"{b}: {s}")

print (get_consensus(statements, weights, my_query_2))

doctor.ai: etesevimab, nirmatrelvir and ritonavir, sotrovimab, elasomeran, tozinameran, molnupiravir, covid-19 vaccine, bamlanivimab, remdesivir, tixagevimab and cilgavimab
kendra: remdesivir, nirmatrelvir and ritonavir
GPT-3: the main drugs used to treat covid-19 are antiviral medications, such as remdesivir and monoclonal antibodies. other drugs being studied to treat severe covid-19 cases include corticosteroids, antiviral drugs, interferon, and anticoagulants.
The answer is a list
0 | ["etesevimab", "nirmatrelvir and ritonavir", "sotrovimab", "elasomeran", "tozinameran", "molnupiravir", "covid-19 vaccine", "bamlanivimab", "remdesivir", "tixagevimab", "cilgavimab"]
1 | ["remdesivir", "nirmatrelvir and ritonavir"]
2 | ["remdesivir", "monoclonal antibodies", "corticosteroids", "antiviral drugs", "interferon", "anticoagulants"]
{'etesevimab': 9, 'nirmatrelvir and ritonavir': 15, 'sotrovimab': 9, 'elasomeran': 9, 'tozinameran': 9, 'molnupiravir': 9, 'covid-19 vaccine': 9, 'bamlanivimab'